<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Working-with-spatial-features" data-toc-modified-id="Working-with-spatial-features-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Working with spatial features</a></span><ul class="toc-item"><li><span><a href="#Map-matching" data-toc-modified-id="Map-matching-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Map matching</a></span></li><li><span><a href="#Rating-information" data-toc-modified-id="Rating-information-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Rating information</a></span></li><li><span><a href="#A-second-example-considering-areas" data-toc-modified-id="A-second-example-considering-areas-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>A second example considering areas</a></span></li></ul></li></ul></div>

# Working with spatial features
Available data is not necessary useful, but need further modelling to be meaningful. 

In [ ]:
import pandas as pd
import geopandas as gpd
import os
import matplotlib.pyplot as plt 
import contextily
import osmnx as ox
from IPython.display import Image

%matplotlib inline
ox.config(log_console=True)
ox.__version__

First, we need to load a file, here one, that contains points in forest. When plotting the file, the visualization does not say much.

In [ ]:
kn_ac = gpd.read_file(r'./data/knotenpunkte-wald_ac.geojson')
kn_ac.head()

In [ ]:
kn_ac.plot()

In [ ]:
kn_ac = kn_ac.to_crs("EPSG:4326")
kn_ac.head()

## Map matching
First, we need to query all the points of interest (POI) within the area our points cover. Next we require a polygon, that covers all the node ('Knotenpunkte'). Convex Hull is the smallest polygon that covers all points in the set. Based this polygon we then use OpenStreeMap data to search for specific information. Points of interest within a specific domain, here parking.

In [ ]:
kn_ch = kn_ac.unary_union.convex_hull
kn_ch

In [ ]:
%%time

pois = ox.geometries_from_polygon(
    kn_ch, tags={"amenity": 'parking'}
)
kn_ac = kn_ac.to_crs("EPSG:3857")
pois = pois.to_crs("EPSG:3857")


In [ ]:
pois.groupby('amenity').amenity.count()

In [ ]:
f,ax = plt.subplots(1)
kn_ac.plot(ax=ax, marker='.')
pois.plot(ax=ax, color='r')
contextily.add_basemap(
    ax, 
    crs=kn_ac.crs.to_string(), 
    source=contextily.providers.Stamen.Toner
)

In [ ]:
kn_ac.crs

In [ ]:
pois.crs

## Rating information
So far the information does not say much, it just gives us information about locations. To rate the information what is more or less valuable, we need to clarify the conditions. A node ('Knotenpunkt') is high rated when there is a lot of parking within the 500 m. 

In [ ]:
kn_albers = kn_ac.to_crs(epsg=3311)
kn_albers.head()

In [ ]:
pois_albers = pois.to_crs(epsg=3311)
pois_albers.head()

In [ ]:
kn_albers['buffer_500m'] = kn_albers.buffer(500)
kn_ac.head()

In [ ]:
joined = gpd.sjoin(
    pois_albers,
    kn_albers.set_geometry('buffer_500m')[['id', 'buffer_500m']],
    op="within"
)

In [ ]:
poi_count = joined.groupby("id")\
                  ["osmid"]\
                  .count()\
                  .to_frame('poi_count')
poi_count.head(5)

In [ ]:
kn_w_counts = kn_albers.merge(
    poi_count, left_on='id', right_index=True
).fillna({"poi_count": 0})

In [ ]:
f, ax = plt.subplots(1, figsize=(9, 9))
kn_w_counts.plot(column="poi_count",
                      scheme="quantiles",
                      alpha=0.5,
                      legend=True,
                      ax=ax
                     )
contextily.add_basemap(ax, 
                       crs=kn_albers.crs.to_string(), 
                       source=contextily.providers.Stamen.Toner
                      )

## A second example considering areas
We do another example using the area 'Umweltzone' which contains the Aachen city centre.

In [ ]:
uz = gpd.read_file("./data/umweltzone.shp")
uz.head()

In [ ]:
uz.plot()

In [ ]:
uz = uz.to_crs("EPSG:4326")
uz.head()

In [ ]:
uz_ch = uz.unary_union.convex_hull
uz_ch

In [ ]:
%%time
pois = ox.geometries_from_polygon(
    uz_ch, tags={"amenity": 'parking'}
)
uz = uz.to_crs("EPSG:3857")
pois = pois.to_crs("EPSG:3857")

In [ ]:
pois.groupby('amenity').amenity.count()

In [ ]:
f,ax = plt.subplots(1,figsize=(12, 12))
uz.plot(ax=ax, marker='.')
pois.plot(ax=ax, color='r')
contextily.add_basemap(
    ax, 
    crs=uz.crs.to_string(), 
    source=contextily.providers.Stamen.Toner
)